# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Leveraging Apache Cassandra. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Boilerplate blocks that creates a (cluster + Keyspace) and sets the keyspace for us to start interacting with databases.

#### Creating a Cluster
The cluster is a collection of nodes that represents a single system. A cluster in Cassandra is one of the shells in the whole Cassandra database. In this case we create a single cluster on our local machine using our local IP.

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace
A keyspace in Cassandra is a namespace that defines data replication on nodes. A cluster contains one keyspace per node. If we need to increase our 
<img src="images/class_table.jpg">

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace
the keyspace in Cassandra is equivalent to a database in an SQL database. Set the default keyspace for all queries made through this Session. This operation blocks until complete.

In [ ]:

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1: The Session data
since we need to filter for `sessionId` and `itemInSession`, this is a good starting point to form our PRIMARY KEY. 

In [ ]:
# store the CREATE statement in 'create_q1' for readability and cleaner implementation.
create_q1 = """CREATE TABLE IF NOT EXISTS session_data (sessionId int, 
                                                        itemInSession int,
                                                        artist text,
                                                        song text,
                                                        length float,
                                                        PRIMARY KEY (sessionId, itemInSession))"""

session.execute(create_q1) # execute the query

In [ ]:
# This block of code is used to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_data (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
# query that will return the session data that we are in search for.
q1 = ("""SELECT sessionId, itemInSession, artist, song 
             FROM session_data
                 WHERE sessionId = 338 AND itemInSession = 4""")

# try block that executes the query
try:
    rows = session.execute(q1)
except Exception as e:
    print(e)

# print all rows of data that match our query    
for row in rows:
    print (row)

## Query 2: The User data
since we need to filter for `userId` and `itemInSession`, this is a good starting point to form our PRIMARY KEY. Additionally, the ask was to return the table sorted by `itemInSession`. This means we will need to add this column as the CLUSTERING COLUMN. These store data in ascending or descending order within the partition for the fast retrieval of similar values. All the fields together are the primary key.

In [ ]:
# store the CREATE statement in a variable
create_q2 = """CREATE TABLE IF NOT EXISTS user_data (userId int,
                                                     sessionId int,
                                                     itemInSession int,
                                                     artist text, 
                                                     song text,
                                                     firstName text,
                                                     lastName text,
                                                     PRIMARY KEY ((userId, sessionId), itemInSession))"""
# Run the Query
try:
    session.execute(create_q2)
except Exception as e:
    print(e)
                    

In [ ]:
file = 'event_datafile_new.csv'

# This block of code is used to set up the CSV file.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_data (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
# query that will return the session data that we are in search for.
q2 = ("""SELECT artist, song, firstName, LastName
             FROM user_data
                 WHERE userid = 10 AND sessionid = 182""")

# try block that executes the query
try:
    rows = session.execute(q2)
except Exception as e:
    print(e)

# print all rows of data that match our query        
for row in rows:
    print (row)

## Query 3: The Song data
since we need to filter for `song`, this is a good starting point to form our PRIMARY KEY. However, adding `userId` can help uniqely identify the song data. Another COMPOSITE KEY is needed here.

In [ ]:
# store the CREATE statement in a variable
create_q3 = """CREATE TABLE IF NOT EXISTS song_data (userId int,
                                                     song text,
                                                     firstName text,
                                                     lastName text,
                                                     PRIMARY KEY (song, userId))"""
# Run the Query
try:
    session.execute(create_q3)
except Exception as e:
    print(e)    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data (userId, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[5]))

In [ ]:
# query that will return the session data that we are in search for.
q3 = ("""SELECT firstName, lastName, song
             FROM song_data
                 WHERE song = 'All Hands Against His Own'""")

# try block that executes the query
try:
    rows = session.execute(q3)
except Exception as e:
    print(e)

# print all rows of data that match our query      
for row in rows:
    print (row)

### Drop the tables before closing out the sessions

Drop tables

In [ ]:
# DROP statements put into a list to iterate in a for loop.
drop_q1 = "DROP TABLE IF EXISTS session_data"
drop_q2 = "DROP TABLE IF EXISTS user_data"
drop_q3 = "DROP TABLE IF EXISTS song_data"
drop_table_queries = [drop_q1, drop_q2, drop_q3]

try:
    for q in drop_table_queries:
        session.execute(q)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()